## Графовые нейронные сети (GNNs)

Графовые нейронные сети представляют собой класс нейронных сетей, специально разработанный для работы с графовыми структурами данных. Они могут моделировать зависимости и взаимодействия между сущностями в графе, позволяя учитывать контекст и структуру данных при выполнении задач.

В GNN каждая вершина графа обновляется на основе её соседей, что позволяет модели учить представления вершин, учитывая их окружение. Такие модели широко применяются в задачах, связанных с анализом социальных сетей, биологических сетей, рекомендательных систем и графовых знаний.

## Graph embedding
Эмбеддинги в контексте графовых нейронных сетей представляют собой векторные представления вершин графа. Они извлекаются из GNN и представляют сущности в пространстве низкой размерности. Эти векторы обладают свойством сохранять структурную информацию о графе и могут использоваться для различных задач, таких как предсказание отношений или классификация вершин.

## Тестирование и Валидация

Тестирование и валидация важны для оценки работы графовых нейронных сетей.

Тестирование позволяет оценить обобщающую способность модели на новых данных, которые не участвовали в обучении. Это важно для проверки, насколько хорошо модель обобщает свои знания на новые сценарии.

Валидация используется для подбора оптимальных гиперпараметров модели. Настройка параметров происходит на основе производительности модели на отложенной валидационной выборке.

### Установка зависимостей

Для работы с графовыми нейронными сетями будем использовать [Pykeen](https://pykeen.readthedocs.io/en/stable/). Необходимо импортировать соответствующую библиотеку. В данном шаге мы устанавливаем Pykeen. Этот шаг включает установку библиотеки Pykeen с использованием команды !pip install pykeen. Pykeen предоставляет реализацию различных моделей графовых знаний и инструменты для работы с ними.

In [1]:
# Установка Pykeen
!pip install pykeen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.3/739.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 28.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.6 MB/s eta 0:00:00
  Created wheel for rexmex: filename=rexmex-0.1.3-py3-none-any.whl size=19952 sha256=077c0b803202f6e420db94805c1ff7338e1e6ca1aaacd725ea013045e0d83068
  Stored in directory: /root/.cache/pip/wheels/13/31/9d/51fb64454e6012bc88dd72b57609646432e478ba35772736d6
Successfully built rexmex


Далее, мы импортируем библиотеку Pykeen для использования её функциональности в нашем коде.

In [2]:
import pykeen

После выполнения этого шага, мы готовы к использованию Pykeen для создания, обучения и оценки моделей графовых знаний.

## Взаимодействие с графовым датасетом и описание данных

В данной работе мы будем использовать графовый датасет [Nations](https://paperswithcode.com/dataset/nations), предоставляющий информацию о связях между стран и их политических отношений. Этот небольшой датасет хорошо подходит для погружения в задачу предсказания отношений между сущностями.


Импорт необходимого модуля Pykeen. Мы импортируем модуль datasets из Pykeen для работы с графовыми датасетами.

In [14]:
from pykeen.datasets import Countries

Загружаем графовый датасет Nations из Pykeen. В результате выполнения этой команды будет создан объект, представляющий графовый датасет.

In [27]:
# Загрузка графового датасета Countries
countries_dataset = Countries()

Мы можем вывести описание датасета, чтобы получить информацию о количестве сущностей, отношений и других характеристиках.

In [28]:
# Просмотр описания датасета
print(countries_dataset)

Countries(training_path="/root/.data/pykeen/datasets/countries/train.txt", testing_path="/root/.data/pykeen/datasets/countries/test.txt", validation_path="/root/.data/pykeen/datasets/countries/valid.txt")


#### Получение тренировочных данных
Мы можем получить тренировочные факты, представленные в виде троек сущность-отношение-сущность.
Таким образом, после выполнения этих шагов, у нас есть загруженный графовый датасет, и мы можем использовать его для обучения и тестирования моделей графовых знаний в Pykeen.

In [29]:
# Получение тренировочных фактов (троек сущность-отношение-сущность)
training_data = countries_dataset.training.mapped_triples
print(training_data[:5])

tensor([[  0,   0,  13],
        [  0,   0, 223],
        [  0,   1,  50],
        [  0,   1, 108],
        [  0,   1, 178]])


## Описание базовой задачи

В данном контексте базовой задачей является предсказание отношений между сущностями в графовом датасете. Каждая тройка в графе представляет отношение между двумя сущностями. Например, в социальной сети, где сущности - пользователи и страницы, отношения могут быть "пользователь подписан на страницу".

In [19]:
# Импорт необходимых модулей Pykeen
from pykeen.pipeline import pipeline


#### Выбор модели
Мы выбираем модель для решения задачи предсказания отношений. В данном случае, мы используем модель [TransE](https://paperswithcode.com/method/transe), одну из популярных моделей для этой задачи.

#### Выбор датасета
Мы указываем графовый датасет, с которым будем работать. В данном примере - nations_dataset, который содержит данные о связях в между странами.

#### Количество эпох обучения
Задаем количество эпох обучения (итераций по всему датасету).

Проверяем наличие GPU устройства и вызываем функцию pipeline(), которая запускает процесс обучения модели, тестирования и валидации.

In [30]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [31]:
# Задаем параметры для pipeline
pipeline_results = pipeline(
    model="TransE",      # Выбираем модель (TransE - одна из моделей для предсказания отношений)
    dataset=countries_dataset,     # Выбираем графовый датасет (Nations)
    training_kwargs=dict(num_epochs=20),   # Количество эпох обучения
    device=device
)

INFO:pykeen.pipeline.api:Using device: cuda


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/5 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/5 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/5 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/5 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/5 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=24.


Evaluating on cuda:0:   0%|          | 0.00/24.0 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.10s seconds



Этот код инициирует обучение модели на выбранном датасете, используя выбранную архитектуру (TransE) и указанные параметры. После завершения обучения, результаты выводятся, и можно провести анализ производительности модели.


Печатаем результаты, которые могут включать в себя метрики производительности, потери на тренировочных данных, а также результаты на тестовом и валидационном наборах данных.

In [32]:
# Выводим результаты
print(pipeline_results)

PipelineResult(random_seed=2197223015, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(271, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(2, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=271, num_relations=2, create_inverse_triples=False, num_triples=1110, path="/root/.data/pykeen/datasets/countries/train.txt"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7e3c541d51b0>, losses=[1.2008440256118775, 1.1004769802093506, 1.0702728271484374, 1.0134424328804017, 1.0079009890556336, 0.8976886749267579, 0.9259099125862121, 0.8967313408851624, 0.9058353781700135, 0.862373161315918, 0.8130576848983765, 0.7769090414047242, 0.786702835559845, 0.7666085720062256, 0.7167340517044067, 0.7002558469772339, 0.6955460071

### Triple Scoring

Импортируем [predict_triples](https://pykeen.readthedocs.io/en/stable/api/pykeen.predict.predict_triples.html)

In [33]:
from pykeen.predict import predict_triples

Рассчитаем оценки для всех validation троек из набора данных, на котором мы обучали модель.

In [34]:
pack = predict_triples(model=pipeline_results.model, triples=countries_dataset.validation)
df = pack.process(factory=pipeline_results.training).df



```
# This is formatted as code
```

Показаны тройки, получившие наивысший score

In [35]:
df.nlargest(n=10, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
9,102,honduras,0,locatedin,5,americas,-7.775967
22,221,south_sudan,0,locatedin,1,africa,-7.960452
23,239,timor-leste,0,locatedin,13,asia,-8.074722
12,128,lesotho,0,locatedin,1,africa,-8.705488
5,53,colombia,0,locatedin,5,americas,-8.756319
21,215,somalia,0,locatedin,1,africa,-8.885624
19,195,rwanda,0,locatedin,1,africa,-8.957662
6,63,djibouti,0,locatedin,1,africa,-8.983191
4,37,burundi,0,locatedin,1,africa,-9.018471
13,134,macau,0,locatedin,13,asia,-9.019730


В рамках самостоятельной работы попробуйте обучить и протестировать модель на любом другом датасете доступном на https://pykeen.readthedocs.io/en/stable/reference/datasets.html.